In [2]:
import pyspark.sql.functions
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession \
    .builder \
    .appName("grab_dataframe") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector:10.0.5') \
    .getOrCreate()

22/11/10 11:34:56 WARN Utils: Your hostname, mbp.local resolves to a loopback address: 127.0.0.1; using 192.168.18.6 instead (on interface en0)
22/11/10 11:34:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/Users/tmillar/dev/repo/hacking-spark-with-mongo/venv/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/tmillar/.ivy2/cache
The jars for the packages stored in: /Users/tmillar/.ivy2/jars
org.mongodb.spark#mongo-spark-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-789fcf1e-7951-4e23-af87-ae58e762019f;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector;10.0.5 in central
	found org.mongodb#mongodb-driver-sync;4.5.1 in central
	[4.5.1] org.mongodb#mongodb-driver-sync;[4.5.0,4.5.99)
	found org.mongodb#bson;4.5.1 in central
	found org.mongodb#mongodb-driver-core;4.5.1 in central
:: resolution report :: resolve 1380ms :: artifacts dl 5ms
	:: modules in use:
	org.mongodb#bson;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-core;4.5.1 from central in [default]
	org.mongodb#mongodb-driver-sync;4.5.1 from central in [default]
	org.mongodb.spark#mongo-spark-connector;10.0.5 from central in [default]
	---------------------------------------------------------------------
	|                 

22/11/10 11:34:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/10 11:34:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/11/10 11:34:59 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/11/10 11:34:59 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:
database = "hacking"
collection = "people"
connectionString = "mongodb://127.0.0.1/hacking"

In [6]:
df = spark.read.format("mongodb") \
    .option("database", database) \
    .option("collection", collection) \
    .load()
print(df)

DataFrame[_id: string, age: int, name: string]


In [7]:
df.show()

+--------------------+----+-----+
|                 _id| age| name|
+--------------------+----+-----+
|636d401e920e9fad5...|  30| John|
|636d401e920e9fad5...|  45|  Joe|
|636d401e920e9fad5...|null|Jason|
+--------------------+----+-----+



### indexing [] gets column object

In [8]:
type(df["name"])

pyspark.sql.column.Column

### select method returns a dataframe of that column

In [9]:
type(df.select("name"))

df.select("name").show()

+-----+
| name|
+-----+
| John|
|  Joe|
|Jason|
+-----+



### head method will grab N Rows from the top of the dataframe

In [10]:
df.head(2)

[Row(_id='636d401e920e9fad564fb66b', age=30, name='John'),
 Row(_id='636d401e920e9fad564fb66c', age=45, name='Joe')]

In [11]:
row = df.head(3)[0]

In [39]:
df.select(["name", "age"]).show()

+-----+----+
| name| age|
+-----+----+
|  Joe|null|
| John|  39|
|Jason|  49|
+-----+----+



### withColumn will return a new dataframe with this new column added

In [12]:
df = df.withColumn("double_age", df["age"] * 2)
df.show()

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+



### withColumnRenamed to rename a column

In [13]:
df.withColumnRenamed("age", "new_age")

DataFrame[_id: string, new_age: int, name: string, double_age: int]

### use SQL

In [14]:
df.createOrReplaceTempView("people")

spark.sql("select * from people").show()
spark.sql("select * from people where age > 40").show()
spark.sql("select * from people order by age desc").show()
spark.sql("select * from people order by age asc").show()
spark.sql("select * from people where age is not null").show()
spark.sql("select * from people where age is null").show()

age = 40
spark.sql("select * from people where age > {0}".format(age)).show()

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad

In [15]:
from pyspark.sql.functions import asc, desc

df.filter("age > 40").show()
df.filter("age is not null").show()
df.orderBy(desc("age")).show()
df.orderBy(asc("age")).show()


df.filter("age is not null and length(name) >= 4").orderBy(desc("age")).select("name").show()



+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+---+----+----------+
|                 _id|age|name|double_age|
+--------------------+---+----+----------+
|636d401e920e9fad5...| 30|John|        60|
|636d401e920e9fad5...| 45| Joe|        90|
+--------------------+---+----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|  45|  Joe|        90|
|636d401e920e9fad5...|  30| John|        60|
|636d401e920e9fad5...|null|Jason|      null|
+--------------------+----+-----+----------+

+--------------------+----+-----+----------+
|                 _id| age| name|double_age|
+--------------------+----+-----+----------+
|636d401e920e9fad5...|null|Jason|      null|
|636d401e920e9fad5...|  30| J